In [1]:
!pip install pandas openpyxl xlrd

Defaulting to user installation because normal site-packages is not writeable


In [26]:
import pandas as pd

# --- Step 1: Read Excel and skip top 5 rows ---
file_path = r"E:\OrderProcssing\data\HK Designs\HK_PO08282025.xlsx"
df = pd.read_excel(file_path, skiprows=5)

# --- Step 2: Use the 7th row as header ---
df.columns = df.iloc[0]   # take first data row as header
df = df[1:].reset_index(drop=True)

# --- Step 3: Clean current column names ---
df.columns = df.columns.astype(str).str.strip()

# --- Step 4: Rename ALL columns manually ---
# 👇 Replace these with whatever names you want in the same order as df.columns
new_column_names = [
    "SrNo", "Item#", "VendorItem#", "StyleCode", "SKU#", "ItemSize",
    "OrderQty", "DeliveryDate", "Hallmark", "GCAL", "not needed", "Metal",
    "Tone", "StampInstr", "Customer", "b", "PO#", "Remark",
    "MinMax", "Color", "Quality", "StoneShape", "CustomerProductionInstruction"
]

# If you have *more or fewer columns*, update this list to match exactly.
df.columns = new_column_names[:len(df.columns)]

selected_columns = ['SrNo', 'StyleCode', 'SKU#', 
                    'ItemSize', 'OrderQty','Metal','Tone','PO#','MinMax','Quality','CustomerProductionInstruction']

df_selected = df[selected_columns].copy()

item_size = df_selected.pop('ItemSize')
df_selected.insert(df_selected.columns.get_loc('StyleCode') + 1, 'ItemSize', item_size)

order_qty = df_selected.pop('OrderQty')
df_selected.insert(df_selected.columns.get_loc('ItemSize') + 1, 'OrderQty', order_qty)

df_selected.insert(df_selected.columns.get_loc('OrderQty') + 1, 'OrderItemPcs', value='')  # Initialize with 0 or empty values

metal = df_selected.pop('Metal')
df_selected.insert(df_selected.columns.get_loc('OrderItemPcs') + 1, 'Metal', metal)

tone = df_selected.pop('Tone')
df_selected.insert(df_selected.columns.get_loc('Metal') + 1, 'Tone', tone)

#df.to_csv(r'C:\Users\Admin\Desktop\HKD_PO_Cleaned.csv', index=False)

# --- Step 5: Done ---

df_selected.head()



,SrNo,StyleCode,ItemSize,OrderQty,OrderItemPcs,Metal,Tone,SKU#,PO#,MinMax,Quality,CustomerProductionInstruction
0,1.0,TRB00729EG,US 7,15,,Platinum,W,1754195.0,Round : I/VS2,-,1.5,with SKU tags & Made in India tags
1,2.0,TRB00729EG,US 7,10,,Platinum,W,1754195.0,Round : I/VS2,-,1.5,with SKU tags & Made in India tags
2,3.0,TRB01252EG,US 7,20,,Platinum,W,1754219.0,Round : I/VS2,NaN,0.9,with SKU tags & Made in India tags
3,4.0,RE20869EG,US 7,10,,Platinum,W,1754214.0,OVAL : H/VS2,GCAL cert inscription,2.35,GIA cert inscription on the girdle must be vi...
4,NaN,NaN,NaN,55,,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# FINAL CODE – HK Designs PO Processing
import pandas as pd

# --- Step 1: Read Excel and skip top 5 rows ---
file_path = r"E:\OrderProcssing\data\HK Designs\HK_PO08282025.xlsx"
df = pd.read_excel(file_path, skiprows=5)

# --- Step 2: Use the 7th row as header ---
df.columns = df.iloc[0]   # take first data row as header
df = df[1:].reset_index(drop=True)

# --- Step 3: Clean current column names ---
df.columns = df.columns.astype(str).str.strip()

# --- Step 4: Rename ALL columns manually ---
new_column_names = [
    "SrNo", "Item#", "VendorItem#", "StyleCode", "SKUNo", "ItemSize",
    "OrderQty", "DeliveryDate", "Hallmark", "GCAL", "not needed", "Metal",
    "Tone", "StampInstr", "Customer", "b", "PO#", "Remark",
    "MinMax", "Color", "Quality", "StoneShape", "CustomerProductionInstruction"
]
df.columns = new_column_names[:len(df.columns)]

# --- Step 5: Select relevant columns ---
selected_columns = [
    'SrNo', 'StyleCode', 'SKUNo',
    'ItemSize', 'OrderQty', 'Metal', 'Tone', 'PO#',
    'MinMax', 'Quality', 'CustomerProductionInstruction'
]
df_selected = df[selected_columns].copy()

# ✅ Step 5A: Clean and format ItemSize
def format_item_size(size):
    if pd.isna(size):
        return ""
    s = str(size).strip().upper()
    # Match formats like "US 7", "US7", "7", "7.5"
    if s.startswith("US"):
        num_part = s.replace("US", "").strip()
        try:
            if float(num_part).is_integer():
                return f"US{int(float(num_part)):02d}"  # US07
            else:
                return f"US{float(num_part)}"           # US7.5
        except:
            return s
    else:
        try:
            if float(s).is_integer():
                return f"US{int(float(s)):02d}"
            else:
                return f"US{float(s)}"
        except:
            return s

df_selected["ItemSize"] = df_selected["ItemSize"].apply(format_item_size)

# --- Step 6: Reorder columns ---
item_size = df_selected.pop('ItemSize')
df_selected.insert(df_selected.columns.get_loc('StyleCode') + 1, 'ItemSize', item_size)

order_qty = df_selected.pop('OrderQty')
df_selected.insert(df_selected.columns.get_loc('ItemSize') + 1, 'OrderQty', order_qty)

df_selected.insert(df_selected.columns.get_loc('OrderQty') + 1, 'OrderItemPcs', value=1)

metal = df_selected.pop('Metal')
df_selected.insert(df_selected.columns.get_loc('OrderItemPcs') + 1, 'Metal', metal)

tone = df_selected.pop('Tone')
df_selected.insert(df_selected.columns.get_loc('Metal') + 1, 'Tone', tone)

# --- Step 7: Ask user input ---
user_input = input("Type 'recycled' if this is recycled platinum, else press Enter: ").strip().lower()

# --- Step 8: Map Metal column ---
def map_metal(value):
    if pd.isna(value):
        return value
    val = str(value).upper().strip()
    if "PLATINUM" in val:
        return "PC95Z" if user_input == "recycled" else "PC95"
    return val  # keep original if not platinum

df_selected["Metal"] = df_selected["Metal"].apply(map_metal)

# --- Step 9: Map Tone column based on Metal ---
def map_tone(metal_value, tone_value):
    if pd.isna(metal_value):
        return tone_value  # keep original if metal is empty
    val = str(metal_value).upper().strip()

    # Platinum
    if val == "PC95" or val == "PC95Z":
        return "PT"

    # Gold mappings
    gold_map = {
        "G14W": "W", "G14Y": "Y", "G14P": "P",
        "G18W": "W", "G18Y": "Y", "G18P": "P",
        "G10W": "W", "G10Y": "Y", "G10P": "P",
        "G14WZ": "W", "G14YZ": "Y", "G14PZ": "P",
        "G18WZ": "W", "G18YZ": "Y", "G18PZ": "P",
        "G10WZ": "W", "G10YZ": "Y", "G10PZ": "P"
    }

    if val in gold_map:
        return gold_map[val]

    return tone_value  # keep original if no mapping

df_selected["Tone"] = df_selected.apply(lambda row: map_tone(row["Metal"], row["Tone"]), axis=1)

# --- Step 10: Read cell C3 for ItemPoNo ---
po_value = pd.read_excel(file_path, header=None, engine='openpyxl').iloc[2, 2]
# Row 3 → index 2, Column C → index 2 (0-based)

# Insert new column after 'Tone'
df_selected.insert(
    df_selected.columns.get_loc('Tone') + 1,
    'ItemPoNo',
    po_value
)

# --- Step 11: Ask user input for Priority ---
priority_value = input("Enter value for Priority column: ").strip()

# Columns to add after 'ItemPoNo'
new_cols = {
    "ItemRefNo": "",
    "StockType": "",
    "Priority": priority_value,
    "MakeType": ""
}

insert_pos = df_selected.columns.get_loc("ItemPoNo") + 1
for col_name, col_value in new_cols.items():
    df_selected.insert(insert_pos, col_name, col_value)
    insert_pos += 1

cust_prod = df_selected.pop('CustomerProductionInstruction')
df_selected.insert(df_selected.columns.get_loc('MakeType') + 1, 'CustomerProductionInstruction', cust_prod)

# --- Step 12: Ask user input for OrderGroup ---
order_group_value = input("Enter value for OrderGroup column: ").strip()

df_selected.insert(
    df_selected.columns.get_loc("CustomerProductionInstruction") + 1,
    "OrderGroup",
    order_group_value
)

# --- Step 13: Generate SpecialRemarks ---
def generate_special_remarks(row):
    metal = str(row["Metal"]).upper()
    tone_code = ""

    if "PC95" in metal:
        tone_code = "PW"
    elif metal.startswith(("G10", "G14", "G18")):
        if "W" in metal:
            tone_code = "GW"
        elif "Y" in metal:
            tone_code = "GY"
        elif "P" in metal:
            tone_code = "GP"

    po_val = str(row["PO#"]) if "PO#" in row else ""
    if "/" in po_val:
        po_suffix = po_val.split("/", 1)[1]
    else:
        po_suffix = ""

    return f"{row['OrderGroup']},{row['SKUNo']},{row['StyleCode']}-{tone_code}{po_suffix}"

df_selected.insert(
    df_selected.columns.get_loc("CustomerProductionInstruction") + 1,
    "SpecialRemarks",
    df_selected.apply(generate_special_remarks, axis=1)
)

# --- Step 14: DesignProductionInstruction based on Tone ---
def generate_design_instruction(tone_value):
    if pd.isna(tone_value):
        return ""
    tone = str(tone_value).upper().strip()
    if tone == "W":
        return "WHITE RODIUM"
    elif tone in ["Y", "P", "PT"]:
        return "NO RODIUM"
    else:
        return ""

df_selected.insert(
    df_selected.columns.get_loc("SpecialRemarks") + 1,
    "DesignProductionInstruction",
    df_selected["Tone"].apply(generate_design_instruction)
)

# --- Step 15: StampInstruction ---
def generate_stamp_instruction(row):
    metal = str(row["Metal"]).upper() if pd.notna(row["Metal"]) else ""
    minmax = str(row["MinMax"]).upper() if pd.notna(row["MinMax"]) else ""
    quality = str(row["Quality"]).upper() if pd.notna(row["Quality"]) else ""

    gcal_text = "GCAL cert inscription" if "GCAL" in minmax else ""

    parts = [metal, "HK LOGO"]
    if gcal_text:
        parts.append(gcal_text)
    if quality:
        parts.append(quality)

    return " + ".join(parts)

df_selected.insert(
    df_selected.columns.get_loc("DesignProductionInstruction") + 1,
    "StampInstruction",
    df_selected.apply(generate_stamp_instruction, axis=1)
)

# --- Step 16: Certificate column after OrderGroup ---
df_selected.insert(
    df_selected.columns.get_loc("OrderGroup") + 1,
    "Certificate",
    ""
)

# --- Step 17: Add 3 columns after 'DeliveryDate' ---
extra_after_delivery = ["DeliveryStatus", "ShippingMethod", "TrackingNo"]
if "DeliveryDate" in df.columns:
    delivery_idx = df.columns.get_loc("DeliveryDate")
    for i, col in enumerate(extra_after_delivery):
        df_selected.insert(delivery_idx + 1 + i, col, "")

# --- Step 18: Add multiple columns after 'Certificate' ---
new_columns_after_certificate = [
    "Basestoneminwt", "Basestonemaxwt", "Basemetalminwt", "Basemetalmaxwt",
    "Productiondeliverydate", "Expecteddeliverydate", "SetPrice", "StoneQuality"
]

insert_pos = df_selected.columns.get_loc("SKUNo") + 1
for col in new_columns_after_certificate:
    df_selected.insert(insert_pos, col, "")
    insert_pos += 1

# --- Step 19: Drop unnecessary columns ---
df_selected.drop(columns=['PO#', 'MinMax', 'Quality'], inplace=True, errors='ignore')

# --- Step 20: Final Preview & Save ---
df_selected.to_csv(r"C:\Users\Admin\Desktop\HKD_PO_Cleaned_test1.csv", index=False)
print("✅ File saved successfully at Desktop: HKD_PO_Cleaned.csv")

df_selected.head(20)


Type 'recycled' if this is recycled platinum, else press Enter:  
Enter value for Priority column:  REG
Enter value for OrderGroup column:  COSTCO


✅ File saved successfully at Desktop: HKD_PO_Cleaned.csv


,SrNo,StyleCode,ItemSize,OrderQty,OrderItemPcs,Metal,Tone,ItemPoNo,DeliveryStatus,ShippingMethod,...,Certificate,SKUNo,Basestoneminwt,Basestonemaxwt,Basemetalminwt,Basemetalmaxwt,Productiondeliverydate,Expecteddeliverydate,SetPrice,StoneQuality
0,1.0,TRB00729EG,US07,15,1,PC95,PT,PO08282025,,,...,,1754195.0,,,,,,,,
1,2.0,TRB00729EG,US07,10,1,PC95,PT,PO08282025,,,...,,1754195.0,,,,,,,,
2,3.0,TRB01252EG,US07,20,1,PC95,PT,PO08282025,,,...,,1754219.0,,,,,,,,
3,4.0,RE20869EG,US07,10,1,PC95,PT,PO08282025,,,...,,1754214.0,,,,,,,,
4,NaN,NaN,,55,1,NaN,NaN,PO08282025,,,...,,NaN,,,,,,,,
